# Model Experimentation
This notebook contains a minimal template for running ML experiments. 

## Imports and Setup

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

import sys
sys.path.insert(0, '../utils')
import model_utils
import geoutils

import logging
import warnings
logging.getLogger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

SEED = 42

%load_ext autoreload
%autoreload 2

## File Locations

In [ ]:
data_dir = "../data/"
input_file = data_dir + '20200501_dataset.csv'
output_dir = "../outputs/"

images_dir = data_dir + 'images/'
indices_dir = data_dir + 'indices/'
pos_mask_dir = data_dir + 'pos_masks/'
neg_mask_dir = data_dir + 'neg_masks/'

areas = ['maicao', 'riohacha', 'uribia', 'arauca', 'cucuta', 'arauquita', 'tibu']

## Load Dataset

In [ ]:
data = pd.read_csv(input_file).reset_index(drop=True).fillna(0)
class_dist = data['target'].value_counts().rename(index=model_utils.VALUE_CODES)

print('Data dimensions: {}'.format(data.shape))
print('\nClass distribution:\n{}\n'.format(class_dist))

data.head()

## Resample Dataset

In [ ]:
neg_dist = {'Formal settlement': (2/5), 'Unoccupied land': (3/5)}
data = model_utils.resample(data, num_neg_samples=50000, neg_dist=neg_dist, random_state=SEED)

class_dist = data['target'].value_counts().rename(index=model_utils.VALUE_CODES)
area_dist = data['area'].value_counts().rename(index=model_utils.AREA_CODES)

print('Data dimensions: {}'.format(data.shape))
print('\nArea distribution:\n{}'.format(area_dist))
print('\nClass distribution:\n{}'.format(class_dist))
print('\nClass distribution (normalized):\n{}\n'.format(class_dist/len(data)))

data.head(3)

## ML Pipeline

In [ ]:
from sklearn.linear_model import LogisticRegression

### Model Training & Evaluation

In [ ]:
label = 'target'
features = [column  for column in data.columns[:-2]]
data[label] = data[label].replace({2:0, 3:0})

splits = data[['area']]
X = data[features]
y = data[label]

print('Class distribution:\n{}'.format(data['target'].value_counts()))

### Nested Spatial Cross Validation

In [ ]:
clf =  LogisticRegression()
param_grid = {'classifier__C':[0.001, 0.01, 1.0, 5.0, 10]}
model_utils.nested_spatial_cv(clf, X, y, splits=splits, param_grid=param_grid, verbose=2);

## Train Model

In [ ]:
clf = LogisticRegression()
X = data[features].fillna(0)
y = data[label]
clf.fit(X, y);

## Prediction

In [ ]:
pd.set_option('use_inf_as_na', True)
area_dict = geoutils.get_filepaths(areas, images_dir, indices_dir, pos_mask_dir, neg_mask_dir)
data = geoutils.read_bands(area_dict, 'uribia')
print('Data dimensions: {}'.format(data.shape))
data.head(3)

In [ ]:
preds = clf.predict_proba(data[features].fillna(0))[:, 1]
preds[(data.iloc[:, :-1].sum(axis=1) == 0)] = -1

geoutils.save_predictions(
    preds, 
    image_src=area_dict['uribia']['images'][0], 
    output_file=output_dir+'20200501_uribia.tiff'
)